# L1: Language modelling

In this lab you will implement and train two neural language models: the fixed-window model and the recurrent neural network model. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [1]:
import torch

For this lab, you should use the GPU if you have one:

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data

The data for this lab is [WikiText](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the “Good” and “Featured” articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the “training” and the “validation” section of the data. We fill this container by reading the corresponding text files. The only processing we do is to whitespace-tokenise, and to enclose each non-empty line within `<bos>` (beginning-of-sentence) and `<eos>` (end-of-sentence) tokens.

In [3]:
class WikiText(object):
    
    def __init__(self):
        self.vocab = {}
        self.train = self.read_data('wiki.train.tokens')
        self.valid = self.read_data('wiki.valid.tokens')
    
    def read_data(self, path):
        ids = []
        with open(path, encoding='utf-8') as source:
            for line in source:
                line = line.rstrip()
                if line:
                    for token in ['<bos>'] + line.split() + ['<eos>']:
                        if token not in self.vocab:
                            self.vocab[token] = len(self.vocab)
                        ids.append(self.vocab[token])
        return ids

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [4]:
wikitext = WikiText()

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.vocab))

Tokens in train: 2099444
Tokens in valid: 218808
Vocabulary size: 33279


## Problem 1: Fixed-window model

In this section, you will implement and train the fixed-window neural language model proposed by [Bengio et al. (2003)](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) and presented in the lectures. Recall that an input to the network takes the form of a vector of $n-1$ integers representing the preceding words. Each integer is mapped to a vector via an embedding layer. (All positions share the same embedding.) The embedding vectors are then concatenated and sent through a two-layer feed-forward network with a non-linearity in the form of a rectified linear unit (ReLU) and a final softmax layer.

### Problem 1.1: Vectorise the data (1&nbsp;point)

Your first task is to write code for transforming the data in the WikiText container into a vectorised form that can be fed to the fixed-window model. Concretely, you will implement a [collate function](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) in the form of a callable vectoriser object. Complete the skeleton code in the cell below:

In [15]:
class FixedWindowVectorizer(object):
    def __init__(self, n):
        self.n = n

    def __call__(self, data):
        x, y = [], []
        for i in range(len(data) - self.n):
            x.append(data[i:i+self.n-1])
            y.append(data[i+self.n-1])
        
        return torch.LongTensor(x), torch.LongTensor(y)

Your code should implement the following specification:

**__init__** (*self*, *n*)

> Creates a new vectoriser with n-gram order $n$. Your code should be able to handle arbitrary n-gram orders $n \geq 1$.

**__call__** (*self*, *data*)

> Transforms WikiText *data* (a list of word ids) into a pair of tensors $\mathbf{X}$, $\mathbf{y}$ that can be used to train the fixed-window model. Let $N$ be the total number of $n$-grams from the token list; then $\mathbf{X}$ is a matrix with shape $(N, n-1)$ and $\mathbf{y}$ is a vector with length $N$.

#### 🤞 Test your code

Test your implementation by running the code in the next cell. Does the output match your expectation?

Yes, it seems reasonable: [Amount of words, Columns]

In [16]:
valid_x, valid_y = FixedWindowVectorizer(3)(wikitext.valid)

print(valid_x.size(), valid_y.size())

torch.Size([218805, 2]) torch.Size([218805])


### Problem 1.2: Implement the model (2&nbsp;points)

Your next task is to implement the fixed-window model based on the graphical specification given in the lecture.

In [17]:
import torch.nn as nn

class FixedWindowModel(nn.Module):

    def __init__(self, n, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.relu = nn.ReLU()
        self.m1 = nn.Linear(embedding_dim * (n-1), hidden_dim)
        self.m2 = nn.Linear(hidden_dim, n_words)

    def forward(self, x):
        embedding = self.embedding(x)
        concat = embedding.view(embedding.size(0), -1)

        l1 = self.m1.forward(concat)
        relu = self.relu.forward(l1)

        return self.m2.forward(relu)

Here is the specification of the two methods:

**__init__** (*self*, *n*, *n_words*, *embedding_dim*=50, *hidden_dim*=50)

> Creates a new fixed-window neural language model. The argument *n* specifies the model&rsquo;s $n$-gram order. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the hidden layer of the feedforward network, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, n-1)$, where $B$ is the batch size. The output of the forward pass is a tensor of shape $(B, V)$ where $V$ is the number of words in the vocabulary.

**Hint:** The most efficient way to implement the vector concatenation in this model is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

In [18]:
model = FixedWindowModel(3, len(wikitext.vocab))
input = torch.randint(0, len(wikitext.vocab), (4, 3-1))
output = model(input)
print(output)
print(output.shape)

tensor([[-0.6790, -0.7814,  0.4275,  ...,  0.3152,  0.0443,  0.1056],
        [-0.2560, -0.9945,  0.1252,  ...,  0.1029, -0.7593, -0.0491],
        [-0.1029, -0.2999, -0.1129,  ..., -0.3775, -0.1736, -0.0282],
        [-0.3420, -0.4455,  0.3824,  ...,  0.0244, -0.0155, -0.0597]],
       grad_fn=<AddmmBackward0>)
torch.Size([4, 33279])


### Problem 1.3: Train the model (3&nbsp;points)

Your final task is to write code to train the fixed-window model using minibatch gradient descent and the cross-entropy loss function. This should be a straightforward generalisation of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [19]:
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

def train_fixed_window(n, n_epochs=1, batch_size=3200, lr=1e-2):
    vectorizer = FixedWindowVectorizer(n)
    n_grams_tensor, targets_tensor = vectorizer(wikitext.train)
    
    ds = TensorDataset(n_grams_tensor, targets_tensor)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)
    
    model = FixedWindowModel(n, len(wikitext.vocab))
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for batch, (input, target) in enumerate(dl):
            optimizer.zero_grad()
            output = model(input)
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {loss}")

    return model

Here is the specification of the training function:

**train_fixed_window** (*n*, *n_epochs* = 1, *batch_size* = 3200, *lr* = 0.01)

> Trains a fixed-window neural language model of order *n* using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

The code in the cell below trains a bigram model.

In [20]:
model_fixed_window = train_fixed_window(2)

Epoch 1, Loss: 10.447539329528809
Epoch 1, Loss: 10.265493392944336
Epoch 1, Loss: 10.10720443725586
Epoch 1, Loss: 9.889732360839844
Epoch 1, Loss: 9.6376314163208
Epoch 1, Loss: 9.412520408630371
Epoch 1, Loss: 9.102190971374512
Epoch 1, Loss: 8.81653881072998
Epoch 1, Loss: 8.464323997497559
Epoch 1, Loss: 8.233697891235352
Epoch 1, Loss: 8.04712200164795
Epoch 1, Loss: 7.801574230194092
Epoch 1, Loss: 7.739325046539307
Epoch 1, Loss: 7.605767726898193
Epoch 1, Loss: 7.545498847961426
Epoch 1, Loss: 7.526491641998291
Epoch 1, Loss: 7.423245429992676
Epoch 1, Loss: 7.438287258148193
Epoch 1, Loss: 7.446885585784912
Epoch 1, Loss: 7.229795455932617
Epoch 1, Loss: 7.031637668609619
Epoch 1, Loss: 7.133314609527588
Epoch 1, Loss: 7.1088643074035645
Epoch 1, Loss: 7.098251819610596
Epoch 1, Loss: 7.017299652099609
Epoch 1, Loss: 6.98715877532959
Epoch 1, Loss: 7.0421576499938965
Epoch 1, Loss: 6.865844249725342
Epoch 1, Loss: 6.993980884552002
Epoch 1, Loss: 6.919534206390381
Epoch 1, Lo

#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 350.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

⚠️ Computing the validation perplexity in one go (for the full validation set) will probably exhaust your computer’s memory and/or take a lot of time. If you run into this problem, do the computation at the minibatch level and aggregate the results.

#### 🤞 Test your code

To see whether your network is learning something, print or plot the loss and/or the perplexity on the training data. If the two values do not decrease during training, try to find the problem before wasting time (and electricity) on useless computation.

Training and even evaluation will take some time – on a CPU, you should expect several minutes per epoch, depending on hardware. Our reference implementation uses a GPU and runs in less than 30 seconds per epoch on [Colab](http://colab.research.google.com).

In [21]:
def calc_perplexity(model, n, batch_size=3200):
    model.eval()

    total_prob, words = 0, 0

    vectorizer = FixedWindowVectorizer(n)
    n_grams_tensor, targets_tensor = vectorizer(wikitext.train)
    
    ds = TensorDataset(n_grams_tensor, targets_tensor)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

    with torch.no_grad():
        for n_grams, targets in dl:
            outputs = model(n_grams)
            log_probs = nn.functional.log_softmax(outputs, dim=1)
            for i, target in enumerate(targets):
                w_log_prob = log_probs[i, target]
                total_prob += w_log_prob.item()
            words += targets.size(0)

    average_prob = total_prob / words
    average_prob_t = torch.tensor(average_prob)
    perplexity = torch.exp(-average_prob_t)
    return perplexity.item()

perplexity = calc_perplexity(model_fixed_window, 2)
print(perplexity)

278.3174133300781


## Problem 2: Recurrent neural network model

In this section, you will implement the recurrent neural network language model. Recall that an input to this model is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer representing the probability distribution over the words at position $i+1$. In theory, the input vector could represent the complete training data; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*. This length is called the **backpropagation-through-time horizon**.

### Problem 2.1: Vectorise the data (1&nbsp;point)

As in the previous problem, your first task is to transform the data in the WikiText container into a vectorised form that can be fed to the model.

In [22]:
class RNNVectorizer(object):
    def __init__(self, bptt_len):
        self.bptt_len = bptt_len

    def __call__(self, data):
        x, y = [], []

        n = len(data) - self.bptt_len

        for i in range(0, n, self.bptt_len):
            m = i + self.bptt_len
            x.append(data[i:m])
            y.append(data[i+1:m + 1])
        
        return torch.LongTensor(x).to(device), torch.LongTensor(y).to(device)

Your vectoriser should meet the following specification:

**__init__** (*self*, *bptt_len*)

> Creates a new vectoriser. The parameter *bptt_len* specifies the backpropagation-through-time horizon.

**__call__** (*self*, *data*)

> Transforms a list of token indexes *data* into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [23]:
valid_x, valid_y = RNNVectorizer(32)(wikitext.valid)

print(valid_x.size(), valid_y.size())

torch.Size([6837, 32]) torch.Size([6837, 32])


### Problem 2.2: Implement the model (2&nbsp;points)

Your next task is to implement the recurrent neural network model based on the graphical specification.

In [24]:
import torch.nn as nn

class RNNModel(nn.Module):
    
    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim).to(device)
        self.lm = nn.Linear(hidden_dim, n_words).to(device)

    def forward(self, x):
        x = self.embedding(x).to(device)
        x = self.lstm(x)[0].to(device)
        return self.lm(x)

Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 50, *hidden_dim* = 50)

> Creates a new recurrent neural network language model based on an LSTM. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

In [25]:
V = len(wikitext.vocab)
rnn_model = RNNModel(V)

dl = DataLoader(wikitext.train, batch_size=32, shuffle=True)
batch = next(iter(dl))

print(rnn_model(batch))

tensor([[-0.0120,  0.0716, -0.1547,  ..., -0.0097, -0.0588, -0.1180],
        [-0.0119,  0.1260, -0.1112,  ...,  0.0556,  0.0170, -0.0228],
        [-0.0641,  0.1058, -0.1053,  ..., -0.0948, -0.0213, -0.0624],
        ...,
        [ 0.0023, -0.1698, -0.3146,  ..., -0.0124,  0.0775,  0.1475],
        [ 0.0180, -0.0797, -0.1701,  ..., -0.0473,  0.0239,  0.1444],
        [-0.0682,  0.0483, -0.0970,  ..., -0.0734, -0.0177, -0.1285]],
       grad_fn=<AddmmBackward0>)


### Problem 2.3: Train the model (3&nbsp;points)

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [29]:
import math

def train_rnn(n_epochs=1, batch_size=1000, bptt_len=32, lr=1e-2):
    dl = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=RNNVectorizer(bptt_len))

    loss_total = 0
    batches = 0

    rnn_model = RNNModel(len(wikitext.vocab)).to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(rnn_model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        for batch, (input, target) in enumerate(dl):
            optimizer.zero_grad()
            output = rnn_model(input)

            output_shaped = output.view(-1, len(wikitext.vocab))
            target_shaped = target.view(-1)
            loss = loss_function(output_shaped, target_shaped)
            loss.backward()
            optimizer.step()

            loss_total += loss
            batches += 1

        print(f"Perplexity: {torch.exp(loss_total/batches)}")

    return rnn_model

Here is the specification of the training function:

**train_rnn** (*n_epochs* = 1, *batch_size* = 1000, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [30]:
model_rnn = train_rnn(2, 1000, 32, 0.01)

Perplexity: 607.0833740234375
Perplexity: 456.4004821777344


#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 280.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

In [31]:
def eval_model(rnn_model, batch_size=1000, bptt_len=32):
    dl = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=RNNVectorizer(bptt_len))
    
    loss_function = nn.CrossEntropyLoss()

    loss_total = 0
    num_samples = 0
    with torch.no_grad():
        for batch, (input, target) in enumerate(dl):
            output = rnn_model(input)

            output_shaped = output.view(-1, len(wikitext.vocab))
            target_shaped = target.view(-1)

            loss_total += loss_function(output_shaped, target_shaped) * batch
            num_samples += batch

    print(f"Final Perplexity: {torch.exp(loss_total/num_samples)}")

    return rnn_model

eval_model(model_rnn)

Final Perplexity: 269.3624572753906


RNNModel(
  (embedding): Embedding(33279, 50)
  (lstm): LSTM(50, 50)
  (lm): Linear(in_features=50, out_features=33279, bias=True)
)

## Problem 3: Parameter initialisation (3&nbsp;points)

The error surfaces explored when training neural networks can be very complex. Because of this, it is important to choose “good” initial values for the parameters. In PyTorch, the weights of the embedding layer are initialised by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the initialisation affects the perplexity of your feed-forward language model. Find research articles that propose different initialisation strategies.

Write a short (150&nbsp;words) report about your experiments and literature search. Use the following prompts:

* What different initialisation did you try? What results did you get?

* How do your results compare to what was suggested by the research articles?

* What did you learn? How, exactly, did you learn it? Why does this learning matter?

You are allowed to consult sources for this problem if you appropriately cite them. If in doubt, please read the [Academic Integrity Policy](https://www.ida.liu.se/~TDDE09/logistics/policies.html#academic-integrity-policy).